In [2]:
import os

In [3]:
%pwd

'/mnt/d/Projects/Project Git repos/AutoVideoSectioning/Research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'/mnt/d/Projects/Project Git repos/AutoVideoSectioning'

In [6]:
import logging
from pathlib import Path
logging.basicConfig(
    # filename='extract_data.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

# English Video

In [21]:
# import torch
# import librosa
# from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, pipeline
# from pydub import AudioSegment
# import moviepy.editor as mp

# # Function to extract audio from video
# def extract_audio_from_video(video_path, audio_path):
#     video = mp.VideoFileClip(str(video_path))  # Convert Path to string
#     video.audio.write_audiofile(str(audio_path))  # Convert Path to string

# # Function to transcribe audio using Hugging Face Transformers in 30-second chunks
# def audio_to_text(audio_path, text_path):
#     device = "cuda:0" if torch.cuda.is_available() else "cpu"
#     processor = Wav2Vec2Processor.from_pretrained("jonatasgrosman/wav2vec2-large-xlsr-53-english")
#     model = Wav2Vec2ForCTC.from_pretrained("jonatasgrosman/wav2vec2-large-xlsr-53-english").to(device)

#     audio = AudioSegment.from_file(audio_path)
#     total_duration = len(audio)  # Total duration in milliseconds
#     # total_duration = 120 * 1000  # 30 seconds in milliseconds
#     chunk_duration = 30 * 1000  # 30 seconds in milliseconds
#     text = ""

#     for i in range(0, total_duration, chunk_duration):
#         chunk = audio[i:i + chunk_duration]
#         chunk.export("temp_chunk.wav", format="wav")
        
#         # Load and preprocess the audio chunk
#         speech_array, sampling_rate = librosa.load("temp_chunk.wav", sr=16_000)
#         inputs = processor(speech_array, sampling_rate=16_000, return_tensors="pt", padding=True)
        
#         with torch.no_grad():
#             logits = model(inputs.input_values.to(device), attention_mask=inputs.attention_mask.to(device)).logits
        
#         predicted_ids = torch.argmax(logits, dim=-1)
#         chunk_text = processor.batch_decode(predicted_ids)[0]
#         text += chunk_text + " "

#     # Write the transcription to the text file
#     with open(text_path, "w", encoding="utf-8") as file:
#         file.write(text.strip())
#     print("Text successfully written to", text_path)

# # Paths
# video_path = Path("Dataset/Videoinput/videoplayback.mp4")
# audio_path = Path("Dataset/AudioExtracted/extracted_audio.wav")
# text_path = Path("Dataset/Transcribe/english_translated_transcribed_text.txt")

# # Extract audio from video
# extract_audio_from_video(video_path, audio_path)

# # Convert audio to text
# audio_to_text(audio_path, text_path)

# print("Text extracted and saved to", text_path)


MoviePy - Writing audio in Dataset/AudioExtracted/extracted_audio.wav


MoviePy - Done.


Some weights of the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-english were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-english and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
Y

Text successfully written to Dataset/Transcribe/english_translated_transcribed_text.txt
Text extracted and saved to Dataset/Transcribe/english_translated_transcribed_text.txt


# with time stamps

In [22]:
import torch
import pandas as pd
import librosa
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from pydub import AudioSegment
import moviepy.editor as mp
from pathlib import Path

# Function to extract audio from video
def extract_audio_from_video(video_path, audio_path):
    video = mp.VideoFileClip(str(video_path))
    video.audio.write_audiofile(str(audio_path))

# Function to transcribe audio using Hugging Face Transformers in 30-second chunks
def audio_to_text(audio_path):
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    processor = Wav2Vec2Processor.from_pretrained("jonatasgrosman/wav2vec2-large-xlsr-53-english")
    model = Wav2Vec2ForCTC.from_pretrained("jonatasgrosman/wav2vec2-large-xlsr-53-english").to(device)

    audio = AudioSegment.from_file(audio_path)
    total_duration = len(audio)
    chunk_duration = 30 * 1000  # 30 seconds in milliseconds
    chunks = []

    for i in range(0, total_duration, chunk_duration):
        chunk = audio[i:i + chunk_duration]
        chunk.export("temp_chunk.wav", format="wav")

        # Load and preprocess the audio chunk
        speech_array, sampling_rate = librosa.load("temp_chunk.wav", sr=16_000)
        inputs = processor(speech_array, sampling_rate=16_000, return_tensors="pt", padding=True)

        with torch.no_grad():
            logits = model(inputs.input_values.to(device), attention_mask=inputs.attention_mask.to(device)).logits

        predicted_ids = torch.argmax(logits, dim=-1)
        chunk_text = processor.batch_decode(predicted_ids)[0]
        chunks.append(chunk_text)

    return chunks

# Paths
# video_path = Path("Dataset/Videoinput/videoplayback.mp4")
video_path = Path("Dataset/Videoinput/MERN JUNE 09-06-2024.mp4")
audio_path = Path("Dataset/AudioExtracted/extracted_audio.wav")
output_excel_path = Path("Dataset/Timestamp/TimestampedText.xlsx")

# Extract audio from video
extract_audio_from_video(video_path, audio_path)

# Convert audio to text
transcriptions = audio_to_text(audio_path)

# Generate DataFrame
chunks_df = pd.DataFrame(transcriptions, columns=['Text'])

# Calculate time intervals
total_duration_secs = len(AudioSegment.from_file(audio_path)) / 1000  # Total duration in seconds
chunk_duration = 30  # Chunk duration in seconds
num_chunks = len(transcriptions)
time_from = [i * chunk_duration for i in range(num_chunks)]
time_to = [(i + 1) * chunk_duration if (i + 1) * chunk_duration < total_duration_secs else total_duration_secs for i in range(num_chunks)]

chunks_df['Time From (s)'] = time_from
chunks_df['Time To (s)'] = time_to

# Reorder columns
chunks_df = chunks_df[['Time From (s)', 'Time To (s)', 'Text']]

# Save DataFrame to Excel
chunks_df.to_excel(output_excel_path, index=False)

print("DataFrame saved to", output_excel_path)


MoviePy - Writing audio in Dataset/AudioExtracted/extracted_audio.wav


MoviePy - Done.


2024-06-12 01:23:41 - WARNING - /mnt/d/Projects/Project Git repos/AutoVideoSectioning/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(

Some weights of the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-english were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClass

DataFrame saved to Dataset/Timestamp/TimestampedText.xlsx


# For Hindi videos

In [23]:
# import torch
# import pandas as pd
# import librosa
# from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
# from pydub import AudioSegment
# import moviepy.editor as mp
# from pathlib import Path
# from googletrans import Translator

# # Function to extract audio from video
# def extract_audio_from_video(video_path, audio_path):
#     video = mp.VideoFileClip(str(video_path))  # Convert Path to string
#     video.audio.write_audiofile(str(audio_path))  # Convert Path to string

# # Function to transcribe audio using Hugging Face Transformers in 30-second chunks
# def audio_to_text(audio_path):
#     device = "cuda:0" if torch.cuda.is_available() else "cpu"
#     processor = Wav2Vec2Processor.from_pretrained("theainerd/Wav2Vec2-large-xlsr-hindi")
#     model = Wav2Vec2ForCTC.from_pretrained("theainerd/Wav2Vec2-large-xlsr-hindi").to(device)

#     audio = AudioSegment.from_file(audio_path)
#     total_duration = len(audio)  # Total duration in milliseconds
#     chunk_duration = 30 * 1000  # 30 seconds in milliseconds
#     chunks = []

#     for i in range(0, total_duration, chunk_duration):
#         chunk = audio[i:i + chunk_duration]
#         chunk.export("temp_chunk.wav", format="wav")
        
#         # Load and preprocess the audio chunk
#         speech_array, sampling_rate = librosa.load("temp_chunk.wav", sr=16_000)
#         inputs = processor(speech_array, sampling_rate=16_000, return_tensors="pt", padding=True)
        
#         with torch.no_grad():
#             logits = model(inputs.input_values.to(device), attention_mask=inputs.attention_mask.to(device)).logits
        
#         predicted_ids = torch.argmax(logits, dim=-1)
#         chunk_text = processor.batch_decode(predicted_ids)[0]
#         chunks.append(chunk_text)

#     return chunks

# # Paths
# video_path = Path("Dataset/Videoinput/MERN JUNE 09-06-2024.mp4")
# audio_path = Path("Dataset/AudioExtracted/extracted_audio.wav")
# output_excel_path = Path("Dataset/Timestamp/TimestampedText.xlsx")

# # Extract audio from video
# extract_audio_from_video(video_path, audio_path)

# # Convert audio to text
# transcriptions = audio_to_text(audio_path)

# # Generate DataFrame
# chunks_df = pd.DataFrame(transcriptions, columns=['Text'])

# # Calculate time intervals
# total_duration_secs = len(AudioSegment.from_file(audio_path)) / 1000  # Total duration in seconds
# chunk_duration = 30  # Chunk duration in seconds
# num_chunks = len(transcriptions)
# time_from = [i * chunk_duration for i in range(num_chunks)]
# time_to = [(i + 1) * chunk_duration if (i + 1) * chunk_duration < total_duration_secs else total_duration_secs for i in range(num_chunks)]

# chunks_df['Time From (s)'] = time_from
# chunks_df['Time To (s)'] = time_to

# # Reorder columns
# chunks_df = chunks_df[['Time From (s)', 'Time To (s)', 'Text']]

# # Save DataFrame to Excel
# chunks_df.to_excel(output_excel_path, index=False)

# print("DataFrame saved to", output_excel_path)


In [24]:
# # Function to translate Hindi text to English in chunks
# def translate_text(hindi_text):
#     translator = Translator()
#     translation = translator.translate(hindi_text, src='hi', dest='en')
#     return translation.text

# # Load DataFrame
# chunks_df = pd.read_excel(output_excel_path)

# # Translate text using df.apply
# chunks_df['Translated Text'] = chunks_df['Text'].apply(translate_text)

# # Save DataFrame with translations to Excel
# chunks_df.to_excel(output_excel_path, index=False)

# print("Translated DataFrame saved to", output_excel_path)


# rewrite by llama2

In [30]:
import pandas as pd
from llama2backend import generate_text
from pathlib import Path

# Define paths
input_excel_path = Path("Dataset/Timestamp/TimestampedText.xlsx")
output_excel_path = Path("Dataset/Timestamp/RewrittenTimestampedText.xlsx")

# Load the DataFrame
chunks_df = pd.read_excel(input_excel_path)

# Process each text cell using the LLM function
def rewrite_text(text):
    prompt = f"""rewrite this Hinglish which is a transcript  proffesional English:
                {text}
             """
    rewritten_text = generate_text(prompt)
    return rewritten_text

# Apply the rewrite_text function to each cell in the 'Text' column
chunks_df['Rewritten_Text'] = chunks_df['Text'].apply(rewrite_text)
# chunks_df['Rewritten_Text'] = chunks_df['Translated Text'].apply(rewrite_text)


# Save the modified DataFrame to a new Excel file
chunks_df.to_excel(output_excel_path, index=False)

print("Rewritten DataFrame saved to", output_excel_path)


Llama.generate: prefix-match hit


 to this:
Hinglish Translation:
Please confirm the chat with Mr. Cool, Tantang, and Nusita. We need to start the meeting as soon as possible. Please ensure that all participants are present and ready to begin. Kindly inform us if there are any issues or delays. Thank you for your cooperation.
Note: In Hinglish, words are often combined or misspelled to create a unique blend of English and Hindi/Urdu vocabulary and grammar.


llama_print_timings:        load time =     373.41 ms
llama_print_timings:      sample time =      35.34 ms /   112 runs   (    0.32 ms per token,  3169.39 tokens per second)
llama_print_timings: prompt eval time =    6679.29 ms /    77 tokens (   86.74 ms per token,    11.53 tokens per second)
llama_print_timings:        eval time =    9360.01 ms /   111 runs   (   84.32 ms per token,    11.86 tokens per second)
llama_print_timings:       total time =   13097.17 ms /   188 tokens
Llama.generate: prefix-match hit


 into this:
                    A simple sign of page icke justing opon his booke wepside mediata istrakik fomta ticket ba prok social on ekiyeto dicnamic nachene ticke ycaffy-styling vogarahisma coloring find tike to yejo cheese japponasive es gemer vany ticke though artesan somersecto jo karkajo dacha





llama_print_timings:        load time =     373.41 ms
llama_print_timings:      sample time =      29.16 ms /   103 runs   (    0.28 ms per token,  3532.84 tokens per second)
llama_print_timings: prompt eval time =    5060.13 ms /   106 tokens (   47.74 ms per token,    20.95 tokens per second)
llama_print_timings:        eval time =    8443.17 ms /   102 runs   (   82.78 ms per token,    12.08 tokens per second)
llama_print_timings:       total time =   13699.80 ms /   208 tokens
Llama.generate: prefix-match hit


...

 *  Thank you. Please ask any questions you have, and I will answer them directly or indirectly. You can also type your questions in chat, and I will respond accordingly. Don't worry about grammar or pronunciation, as we will see each other in the next session or get thank you for being so corporate-like.
I hope this helps! Let me know if you have any further questions.


llama_print_timings:        load time =     373.41 ms
llama_print_timings:      sample time =      24.38 ms /    87 runs   (    0.28 ms per token,  3568.21 tokens per second)
llama_print_timings: prompt eval time =    4344.63 ms /    94 tokens (   46.22 ms per token,    21.64 tokens per second)
llama_print_timings:        eval time =    7228.65 ms /    86 runs   (   84.05 ms per token,    11.90 tokens per second)
llama_print_timings:       total time =   11739.84 ms /   180 tokens
Llama.generate: prefix-match hit


 *  "I am looking for a job as a software engineer."
                  ❓ What kind of job are you looking for?
                  *  "I want to work for a company that develops cutting-edge technology."
                  ❓ Can you tell me more about your experience in software engineering?
                  *  "I have a bachelor's degree in computer science from a top university."
                  ❓ That's impressive! What are your long-term career goals?
                  *  "I want to become a leading expert in my field and make a significant impact on the industry."
                  ❓ How do you plan to achieve that?
                  *  "I am willing to put in the hard work and dedication required to succeed."
                  ❓ Can you give me an example of a project you worked on and how you contributed to it?
                  *  "Yes, I worked on a team project for my computer science course where we developed a mobile app for tracking personal finances."
                  ❓ 


llama_print_timings:        load time =     373.41 ms
llama_print_timings:      sample time =      70.55 ms /   256 runs   (    0.28 ms per token,  3628.68 tokens per second)
llama_print_timings: prompt eval time =     247.82 ms /     5 tokens (   49.56 ms per token,    20.18 tokens per second)
llama_print_timings:        eval time =   21336.44 ms /   255 runs   (   83.67 ms per token,    11.95 tokens per second)
llama_print_timings:       total time =   22097.42 ms /   260 tokens


Rewritten DataFrame saved to Dataset/Timestamp/RewrittenTimestampedText.xlsx


In [31]:
import pandas as pd
from llama2backend import generate_text
from pathlib import Path

# Define paths
input_excel_path = Path("Dataset/Timestamp/RewrittenTimestampedText.xlsx")
output_excel_path = Path("Dataset/Timestamp/TopicsCoveredRewrittenTimestampedText.xlsx")

# Load the DataFrame
chunks_df = pd.read_excel(input_excel_path)

# Function to split text into chunks
def chunk_text(text, max_length=500):
    words = text.split()
    chunks = []
    current_chunk = []

    for word in words:
        if len(" ".join(current_chunk + [word])) <= max_length:
            current_chunk.append(word)
        else:
            chunks.append(" ".join(current_chunk))
            current_chunk = [word]
    if current_chunk:
        chunks.append(" ".join(current_chunk))
    
    return chunks

# Function to extract topics from text chunks
def extract_topics(text):
    text_chunks = chunk_text(text)
    topics = []

    for chunk in text_chunks:
        prompt = f"Just summarize in bulletpoint headings as Topics and subtopics:\n{chunk}"
        chunk_topics = generate_text(prompt)
        topics.append(chunk_topics)

    return " ".join(topics)

# Concatenate 'Rewritten_Text' and 'Topics_Covered' into a new column 'Concatenated_Text'
chunks_df['Concatenated_Text'] = chunks_df.apply(lambda row: f"{row['Rewritten_Text']} Topics Covered: {row['Text']}", axis=1)

# Apply the extract_topics function to each cell in the 'Concatenated_Text' column
chunks_df['Topics_Covered'] = chunks_df['Concatenated_Text'].apply(extract_topics)

# Save the modified DataFrame to a new Excel file
chunks_df.to_excel(output_excel_path, index=False)

print("Rewritten DataFrame with topics covered and concatenated text saved to", output_excel_path)


Llama.generate: prefix-match hit


 tantang nusita start meeting soon all participants present ready begin inform any issues delays thank you cooperation.
Here is a summary of the text in bulletpoint headings:
Topic: Hinglish Translation
Subtopic 1: Confirmation of Chat
* Please confirm the chat with Mr. Cool, Tantang, and Nusita.
Subtopic 2: Starting the Meeting
* We need to start the meeting as soon as possible.
Subtopic 3: Participants' Presence and Readiness
* Kindly ensure that all participants are present and ready to begin.
Subtopic 4: Informing of Issues or Delays
* Please inform us if there are any issues or delays.
Subtopic 5: Thank You for Cooperation
* Thank you for your cooperation.


llama_print_timings:        load time =     373.41 ms
llama_print_timings:      sample time =      50.45 ms /   181 runs   (    0.28 ms per token,  3587.43 tokens per second)
llama_print_timings: prompt eval time =    6743.94 ms /   144 tokens (   46.83 ms per token,    21.35 tokens per second)
llama_print_timings:        eval time =   15195.97 ms /   180 runs   (   84.42 ms per token,    11.85 tokens per second)
llama_print_timings:       total time =   22297.18 ms /   324 tokens
Llama.generate: prefix-match hit


i.
Here is a summary of the text in bulletpoint headings:
Topics:

* Tantanguaritanusito (Tanggua)
	+ Kalam (speech)
	+ Loganit (logical reasoning)
	+ Philebaniti (pleasure)
	+ Hamlok (imagination)
	+ Stemas (thoughts)
* Start (beginning)
	+ Garrat (difficulty)
	+ Though (thought)
	+ Estemalmamloganiga (difficulty in understanding the meaning of the text)
* Fom banata (from the beginning)
	+ Uplotbarunkerketikadumetosabnahuchi (a phrase that is difficult to understand)
Note: The text appears to be a poem or song, and the language used is complex and abstract. It may be helpful to listen to the audio recording of the text to better understand its meaning.


llama_print_timings:        load time =     373.41 ms
llama_print_timings:      sample time =      60.06 ms /   216 runs   (    0.28 ms per token,  3596.58 tokens per second)
llama_print_timings: prompt eval time =    2432.71 ms /    52 tokens (   46.78 ms per token,    21.38 tokens per second)
llama_print_timings:        eval time =   18014.55 ms /   215 runs   (   83.79 ms per token,    11.93 tokens per second)
llama_print_timings:       total time =   20873.18 ms /   267 tokens
Llama.generate: prefix-match hit


...

 "I am committed to continuous learning and professional development, and I am always looking for new opportunities to grow and challenge myself." ❓ That's great to hear! Can you tell me more about your experience with programming languages and technologies? * "I have experience with a variety of programming languages, including Java, Python, and C++, and I am proficient in using Agile methodologies and DevOps tools." ❓ Excellent! What do you think sets you apart from other candidates? * "I bring a unique combination of technical skills and business acumen to the table, which allows me to approach problems from a holistic perspective and deliver solutions that are both innovative and practical." ❓ That's impressive! Can you give me an example of a project you worked on that demonstrates your skills and experience? * "Sure! I led a team in developing a complex software system for a Fortune 500 company, and we were able to deliver the project ahead of schedule and within budget." ❓ Wow, 


llama_print_timings:        load time =     373.41 ms
llama_print_timings:      sample time =      75.31 ms /   256 runs   (    0.29 ms per token,  3399.24 tokens per second)
llama_print_timings: prompt eval time =    6140.76 ms /   131 tokens (   46.88 ms per token,    21.33 tokens per second)
llama_print_timings:        eval time =   21660.40 ms /   255 runs   (   84.94 ms per token,    11.77 tokens per second)
llama_print_timings:       total time =   28317.65 ms /   386 tokens
Llama.generate: prefix-match hit


.com
- Hard work and dedication required to succeed
- Examples of past projects and how they demonstrate your skills and abilities
- Handling challenges and overcoming obstacles
- Communication and collaboration with team members
- Time management and prioritization of tasks
- Adaptability and flexibility in the face of changing requirements or unexpected issues.


llama_print_timings:        load time =     373.41 ms
llama_print_timings:      sample time =      21.02 ms /    77 runs   (    0.27 ms per token,  3662.83 tokens per second)
llama_print_timings: prompt eval time =    4446.84 ms /    96 tokens (   46.32 ms per token,    21.59 tokens per second)
llama_print_timings:        eval time =    6491.05 ms /    77 runs   (   84.30 ms per token,    11.86 tokens per second)
llama_print_timings:       total time =   11077.80 ms /   173 tokens


Rewritten DataFrame with topics covered and concatenated text saved to Dataset/Timestamp/TopicsCoveredRewrittenTimestampedText.xlsx


# Hindi Video

In [13]:
%%time
import torch
import librosa
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, pipeline
from pydub import AudioSegment
import moviepy.editor as mp

# Function to extract audio from video
def extract_audio_from_video(video_path, audio_path):
    video = mp.VideoFileClip(str(video_path))  # Convert Path to string
    video.audio.write_audiofile(str(audio_path))  # Convert Path to string

# Function to transcribe audio using Hugging Face Transformers in 30-second chunks
def audio_to_text(audio_path, text_path):
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    processor = Wav2Vec2Processor.from_pretrained("theainerd/Wav2Vec2-large-xlsr-hindi")
    model = Wav2Vec2ForCTC.from_pretrained("theainerd/Wav2Vec2-large-xlsr-hindi").to(device)

    audio = AudioSegment.from_file(audio_path)
    total_duration = len(audio)  # Total duration in milliseconds
    # total_duration = 10 * 30  * 1000  # 30 seconds in milliseconds
    chunk_duration = 30 * 1000  # 30 seconds in milliseconds
    text = ""

    for i in range(0, total_duration, chunk_duration):
        chunk = audio[i:i + chunk_duration]
        chunk.export("temp_chunk.wav", format="wav")
        
        # Load and preprocess the audio chunk
        speech_array, sampling_rate = librosa.load("temp_chunk.wav", sr=16_000)
        inputs = processor(speech_array, sampling_rate=16_000, return_tensors="pt", padding=True)
        
        with torch.no_grad():
            logits = model(inputs.input_values.to(device), attention_mask=inputs.attention_mask.to(device)).logits
        
        predicted_ids = torch.argmax(logits, dim=-1)
        chunk_text = processor.batch_decode(predicted_ids)[0]
        text += chunk_text + " "

    # Write the transcription to the text file
    with open(text_path, "w", encoding="utf-8") as file:
        file.write(text.strip())
    print("Text successfully written to", text_path)

# Paths
video_path = Path("Dataset/Videoinput/MERN JUNE 09-06-2024.mp4")
audio_path = Path("Dataset/AudioExtracted/extracted_audio.wav")
text_path = Path("Dataset/Transcribe/transcribed_text.txt")

# Extract audio from video
# extract_audio_from_video(video_path, audio_path)

# Convert audio to text
audio_to_text(audio_path, text_path)

print("Text extracted and saved to", text_path)


2024-06-11 20:36:23 - WARNING - /mnt/d/Projects/Project Git repos/AutoVideoSectioning/venv/lib/python3.10/site-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(

Some weights of the model checkpoint at theainerd/Wav2Vec2-large-xlsr-hindi were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoin

Text successfully written to Dataset/Transcribe/transcribed_text.txt
Text extracted and saved to Dataset/Transcribe/transcribed_text.txt
CPU times: user 57.1 s, sys: 1.81 s, total: 58.9 s
Wall time: 52.4 s


In [7]:
# import moviepy.editor as mp
# from pathlib import Path
# import speech_recognition as sr

# # Function to extract audio from video
# def extract_audio_from_video(video_path, audio_path):
#     video = mp.VideoFileClip(str(video_path))  # Convert Path to string
#     video.audio.write_audiofile(str(audio_path))  # Convert Path to string

# # Function to transcribe audio using SpeechRecognition and Google Speech API
# def audio_to_text(audio_path, text_path):
#     r = sr.Recognizer()
    
#     with sr.AudioFile(str(audio_path)) as source:  # Convert Path to string
#         audio_data = r.record(source)
#         try:
#             text = r.recognize_google(audio_data, language='hi-IN')
#             print(f"Recognized text: {text}")

#             with open(text_path, 'w', encoding='utf-8') as file:
#                 file.write(text)
#             print("Text saved to", text_path)

#         except sr.UnknownValueError:
#             print("Google Speech Recognition could not understand the audio")
#         except sr.RequestError as e:
#             print(f"Could not request results from Google Speech Recognition service; {e}")

# # Paths
# video_path = Path("Dataset/Videoinput/MERN JUNE 09-06-2024.mp4")
# audio_path = Path("Dataset/AudioExtracted/extracted_audio.wav")
# text_path = Path("Dataset/Transcribe/transcribed_text.txt")

# # Extract audio from video
# extract_audio_from_video(video_path, audio_path)

# # Convert audio to text
# audio_to_text(audio_path, text_path)

# print("Text extracted and saved to", text_path)


MoviePy - Writing audio in Dataset/AudioExtracted/extracted_audio.wav


MoviePy - Done.
Could not request results from Google Speech Recognition service; recognition connection failed: [Errno 32] Broken pipe
Text extracted and saved to Dataset/Transcribe/transcribed_text.txt


# Hindi to english

In [8]:
# %%time
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# # Paths
# hindi_text_path = Path("Dataset/Transcribe/transcribed_text.txt")
# english_text_path = Path("Dataset/Transcribe/english_translated_transcribed_text.txt")

# # Load the tokenizer and model
# tokenizer = AutoTokenizer.from_pretrained("snehalyelmati/mt5-hindi-to-english")
# model = AutoModelForSeq2SeqLM.from_pretrained("snehalyelmati/mt5-hindi-to-english")

# # Function to translate Hindi text to English in chunks
# def translate_text_in_chunks(hindi_text_path, english_text_path, chunk_size=500):
#     with open(hindi_text_path, "r", encoding="utf-8") as file:
#         hindi_text = file.read()
    
#     # Split text into chunks
#     chunks = [hindi_text[i:i + chunk_size] for i in range(0, len(hindi_text), chunk_size)]
    
#     translated_text = ""
    
#     for chunk in chunks:
#         inputs = tokenizer.encode(chunk, return_tensors="pt", max_length=512, truncation=True)
#         outputs = model.generate(inputs, max_length=512, num_beams=4, early_stopping=True)
#         translated_chunk = tokenizer.decode(outputs[0], skip_special_tokens=True)
#         translated_text += translated_chunk + " "
    
#     # Write the translated text to the text file
#     with open(english_text_path, "w", encoding="utf-8") as file:
#         file.write(translated_text.strip())
    
#     print("Translated text successfully written to", english_text_path)

# # Translate the text
# translate_text_in_chunks(hindi_text_path, english_text_path)


2024-06-11 20:30:17 - WARNING - /mnt/d/Projects/Project Git repos/AutoVideoSectioning/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



Translated text successfully written to Dataset/Transcribe/english_translated_transcribed_text.txt
CPU times: user 1min 3s, sys: 1.61 s, total: 1min 5s
Wall time: 48.5 s


In [14]:
from pathlib import Path
from googletrans import Translator

# Paths
hindi_text_path = Path("Dataset/Transcribe/transcribed_text.txt")
english_text_path = Path("Dataset/Transcribe/english_translated_transcribed_text.txt")

# Initialize the translator
translator = Translator()

# Function to translate Hindi text to English in chunks
def translate_text_in_chunks(hindi_text_path, english_text_path, chunk_size=500):
    with open(hindi_text_path, "r", encoding="utf-8") as file:
        hindi_text = file.read()
    
    # Split text into chunks
    chunks = [hindi_text[i:i + chunk_size] for i in range(0, len(hindi_text), chunk_size)]
    
    translated_text = ""
    
    for chunk in chunks:
        translation = translator.translate(chunk, src='hi', dest='en')
        translated_text += translation.text + " "
    
    # Write the translated text to the text file
    with open(english_text_path, "w", encoding="utf-8") as file:
        file.write(translated_text.strip())
    
    print("Translated text successfully written to", english_text_path)

# Translate the text
translate_text_in_chunks(hindi_text_path, english_text_path)


Translated text successfully written to Dataset/Transcribe/english_translated_transcribed_text.txt
